# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов. Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Обзор данных

Изучим общую информацию из датафрейма. 

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [2]:
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


В двух столбцах есть пропущенные значения. 
При первичном обзоре очевидно, что их доля значительная, поэтому они не могут быть проигнорированы. 
Кроме того, в столбце **days_employed** обнаружена аномалия: отрицательные значения. 

# Предобработка данных

## Заполнение пропусков

Заменим отрицательный значения в **days_employed** на положительные, взяв каждое значение столбца по модулю. Это необходимо сделать до того, как мы возьмум медиану (если это потребуется). 

In [3]:
data['days_employed'] = data['days_employed'].abs()

In [4]:
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Отрицательные значения в **days_employed** заменены корректными положительными. 
В столбцах **days_employed** и **total_income** пропущено несколько тысяч значений. Подсчитаем точное их число.

In [5]:
print(len(data[data['days_employed'].isna()])) 

2174


2174 пропущенные значения - это 10% данных. 
Предположим, что это люди, впервые вышедшие на работу. Изучим первые 5 строк с пропущенным значением **days_employed**:

In [6]:
display(data[data['days_employed'].isna()].head()) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Предположение о том, что пропущены значения только для тех, кто впервые вышел на работу, неверно: в случайную выборку заемщиков попали клиенты от 41 до 65 лет, в том числе 3 пенсионера. Наиболее вероятно то, что данные пропущены по ошибке (случайные или полностью случайные). 

Количественные переменные **days_employed** и **total_income** рационально заменить на характерные значения. 
Таким образом, данные о 10% заемщиков будут учтены при проверке гипотез.
Заменим **NaN** в **days_employed** на медианное значение, чтобы получить наиболее объективную картину: в данных присутствует значительный разброс значений, и учет их влияния (применение среднего арифметического) на новое значение искажал бы итоговый результат. 

In [7]:
days_employed_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(value = days_employed_median)
display(data.head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Проверим, удалось ли полностью заменить пропущенные значения.

In [8]:
print(len(data[data['days_employed'].isna()])) 

0


Аналогично заменим пропуски в количественной переменной **total_income**

In [9]:
print(len(data[data['total_income'].isna()])) 

2174


In [10]:
display(data[data['total_income'].isna()].head()) 

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,2194.220567,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,2194.220567,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,2194.220567,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,2194.220567,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,2194.220567,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу


Не удается проследить закономерность в причинах пропуска **total_income** в случайной выборке заемщиков.
Сделаем замену пропусков на медианное значение в столбце данных. 
Доход разных групп заемщиков **income_type** может отличаться, поэтому заменим пропуски в уровне дохода **total_income** на медианное значение внутри групп заемщиков **income_type**.

In [11]:
data_transformed = data.groupby('income_type')['total_income'].transform('median')
data['total_income'] = data['total_income'].fillna(data_transformed)
display(data.head(10))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверим, удалось ли полностью заменить пропущенные значения.

In [12]:
print(len(data[data['total_income'].isna()])) 

0


Еще раз проверим данные на наличие пропусков

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Все пропуски устранены, а именно: проведена замена в количественных переменных на значение, наиболее объективно отражающее выборку.
Следует заметить, что возможен и консервативный подход к оценке риска надежности заемщиков. 
При таком подходе строки с недостающими данными не учитываются/ недостающие значения заменяются наиболее пессимистичными из выборки (если строки содержат важную информацию, терять которую не хотелось бы). 

Заменим вещественный значения **float** на целочисленные **int** в целях упрощения работы с массивом и оптимизации памяти, затрачиваемой на обработку данных. 

In [14]:
data['days_employed'] = data['days_employed'].astype('int')
data['total_income'] = data['total_income'].astype('int')

In [15]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


## Проверка данных на аномалии и исправления

При изучении исходного датафрейма были обнаружены аномалии в данных: отрицательные значения в **days_employed**.
Проверим исходный датафрейм на другие аномалии. 

In [16]:
print(data['children'].value_counts()) 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


При анализе данных мы обнаружили, что у 47 заемщиков отрицательное число детей. Это явная ошибка. 
Кроме того, 20 заемщиков являются счастливыми родителями 20 детей. Это значение вызывает большие сомнения.
Исправим.

In [17]:
data['children'] = data['children'].abs()
data['children'] = data['children'].replace(to_replace =20, value =2)
print(data['children'].value_counts()) 

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64


Вот теперь звучит правдоподобно.

In [18]:
print(data['days_employed'].value_counts()) 

2194      2180
133         16
327         16
438         15
223         14
          ... 
4507         1
376084       1
349465       1
10636        1
2049         1
Name: days_employed, Length: 9086, dtype: int64


Значения общего трудового стажа в днях **days_employed** выглядит реалистично. 
*Обратить внимание*: часто встречается значение 2194 дней, или чуть более 6 лет. Нет ли у банка программы, которая дает льготы для заемщиков с трудовым стажем 6+ лет?

In [19]:
print(data['dob_years'].unique())

[42 36 33 32 53 27 43 50 35 41 40 65 54 56 26 48 24 21 57 67 28 63 62 47
 34 68 25 31 30 20 49 37 45 61 64 44 52 46 23 38 39 51  0 59 29 60 55 58
 71 22 73 66 69 19 72 70 74 75]


В датафрейме есть заемщики с нулевым возрастом, посчитаем их:

In [20]:
age_below_18 = data[data['dob_years'] == 0]['dob_years'].count()
print(age_below_18)

101


Кто же этот 101 заемщик с нулевым возрастом? 

In [21]:
display(data[data['dob_years'] == 0].head())

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291,автомобиль
149,0,2664,0,среднее,1,в разводе,3,F,сотрудник,0,70176,операции с жильем
270,3,1872,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166,ремонт жилью
578,0,397856,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620,строительство собственной недвижимости
1040,0,1158,0,высшее,0,в разводе,3,F,компаньон,0,303994,свой автомобиль


Заменим нулевой возраст заемщиков на медианный возраст заемщиков

In [22]:
dob_years_median = data['dob_years'].median()
data['dob_years'] = data['dob_years'].replace(to_replace =0, value =dob_years_median)
print(data['dob_years'].unique())

[42. 36. 33. 32. 53. 27. 43. 50. 35. 41. 40. 65. 54. 56. 26. 48. 24. 21.
 57. 67. 28. 63. 62. 47. 34. 68. 25. 31. 30. 20. 49. 37. 45. 61. 64. 44.
 52. 46. 23. 38. 39. 51. 59. 29. 60. 55. 58. 71. 22. 73. 66. 69. 19. 72.
 70. 74. 75.]


Отлично! С возрастом заемщиков разобрались. 

In [23]:
print(data['education'].unique())

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


In [24]:
data['education']=data['education'].str.lower()
print(data['education'].unique())

['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


Мы привели в порядок данные в столбце **education**.

In [25]:
print(data['education_id'].unique()) 
print(data['family_status_id'].unique()) 

[0 1 2 3 4]
[0 1 2 3 4]


Данные в **education_id** и **family_status_id** не требуют предобработки.

In [26]:
print(data['family_status'].unique()) 

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']


In [27]:
data['family_status']=data['family_status'].str.lower()
print(data['family_status'].unique()) 

['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'не женат / не замужем']


Провели незначительное улучшение в оформлении, приведя все значения в единый формат (нижний регистр).

In [28]:
print(data['gender'].unique()) 

['F' 'M' 'XNA']


In [29]:
gender_XNA=data[data['gender']=='XNA']
display(gender_XNA)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
10701,0,2358,24.0,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905,покупка недвижимости


In [30]:
data = data.loc[data['gender'] != 'XNA']

У нас нет возможности с уверенностью определить пол заемщика, удалим одну строку из датафрейма. 

In [31]:
print(data['income_type'].unique()) 

['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']


Данные в **income_type** в порядке. 

In [32]:
print(data['debt'].unique())  

[0 1]


Данные в **debt** так же не требуют исправлений.

In [33]:
print(data['total_income'].value_counts()) 

142594    1105
172357     509
118514     414
150447     147
144533       3
          ... 
205761       1
179138       1
61724        1
99269        1
126820       1
Name: total_income, Length: 18607, dtype: int64


Данные в **income_type** в порядке. 

In [34]:
print(data['purpose'].unique())  

['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


Данные в **purpose** в порядке, но требуют лучшей категоризации.

Датафрейм очищен от аномалий. 

## Удаление дубликатов

In [35]:
print('Дубликатов в таблице:', data.duplicated().sum())

Дубликатов в таблице: 72


In [36]:
duplicated_data = data[data.duplicated()].head()
display(duplicated_data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,2194,41.0,среднее,1,женат / замужем,0,F,сотрудник,0,142594,покупка жилья для семьи
3290,0,2194,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу
4182,1,2194,34.0,высшее,0,гражданский брак,1,F,сотрудник,0,142594,свадьба
4851,0,2194,60.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,свадьба
5557,0,2194,58.0,среднее,1,гражданский брак,1,F,пенсионер,0,118514,сыграть свадьбу


Мы ознакомились с задублированными данными. 

In [37]:
data=data.drop_duplicates().reset_index(drop=True)
data.info()
print('Дубликатов в таблице:', data.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21452 entries, 0 to 21451
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21452 non-null  int64  
 1   days_employed     21452 non-null  int64  
 2   dob_years         21452 non-null  float64
 3   education         21452 non-null  object 
 4   education_id      21452 non-null  int64  
 5   family_status     21452 non-null  object 
 6   family_status_id  21452 non-null  int64  
 7   gender            21452 non-null  object 
 8   income_type       21452 non-null  object 
 9   debt              21452 non-null  int64  
 10  total_income      21452 non-null  int64  
 11  purpose           21452 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB
Дубликатов в таблице: 0


Строки, в которых информация полностью дублировалась **(72)**, удалены.
**Предполагаемые причины** появления дубликатов: ошибка при копировании данных, учет одного заявления дважды.
Использован **метод удаления дубликатов** с изменением индексов и удалением старых, так как исходные задублированные данные для анализа не потребуются. 

## Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма

In [38]:
data_education=data[['education_id','education']]
display(data_education)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
...,...,...
21447,1,среднее
21448,1,среднее
21449,1,среднее
21450,1,среднее


In [39]:
data_family=data[['family_status_id','family_status']]
display(data_family)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
...,...,...
21447,1,гражданский брак
21448,0,женат / замужем
21449,1,гражданский брак
21450,0,женат / замужем


Два новых словаря созданы. Мы сможем обращаться к ним по идентификатору в дальнейшем. 

In [40]:
data_education=data[['education_id','education']]
data_education=data_education.drop_duplicates().reset_index(drop=True)
display(data_education)

,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [41]:
data_family=data[['family_status_id','family_status']]
data_family=data_family.drop_duplicates().reset_index(drop=True)
display(data_family)

,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


Текстовые описания столбцов **family_status** и **education** дублируются соответствующими идентификаторами. Удалим их.

In [42]:
data = data.drop(columns=['family_status', 'education'])
display(data)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...
21447,1,4529,43.0,1,1,F,компаньон,0,224791,операции с жильем
21448,0,343937,67.0,1,0,F,пенсионер,0,155999,сделка с автомобилем
21449,1,2113,38.0,1,1,M,сотрудник,1,89672,недвижимость
21450,3,3112,38.0,1,0,M,сотрудник,1,244093,на покупку своего автомобиля


## Категоризация дохода

In [43]:
def total_income_category(total_income):
    if total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    return 'A'

print(total_income_category(25000))
print(total_income_category(235000))

E
B


Мы убедились в том, что функция корректно распределяет заемщиков с разным уровнем дохода по категориям. Добавим новый столбец к исходному датафрейму. 

In [44]:
data['total_income_category']=data['total_income'].apply(total_income_category)
display(data.head(10))

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42.0,0,0,F,сотрудник,0,253875,покупка жилья,B
1,1,4024,36.0,1,0,F,сотрудник,0,112080,приобретение автомобиля,C
2,0,5623,33.0,1,0,M,сотрудник,0,145885,покупка жилья,C
3,3,4124,32.0,1,0,M,сотрудник,0,267628,дополнительное образование,B
4,0,340266,53.0,1,1,F,пенсионер,0,158616,сыграть свадьбу,C
5,0,926,27.0,0,1,M,компаньон,0,255763,покупка жилья,B
6,0,2879,43.0,0,0,F,компаньон,0,240525,операции с жильем,B
7,0,152,50.0,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35.0,0,1,F,сотрудник,0,95856,на проведение свадьбы,C
9,0,2188,41.0,1,0,M,сотрудник,0,144425,покупка жилья для семьи,C


Отлично! Мы корректно добавили новый столбец к датафрейму. 

Категоризируем **purpose**: при текущем формате записи данные неудобно группировать. 

In [45]:
def purpose_group(purpose):
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'жиль' in purpose or 'недвижимо' in purpose:
        return 'операции с недвижимостью'
    elif 'автомобил' in purpose:
        return 'операции с автомобилем'
    elif 'образовани'in purpose:
        return 'получение образования'
    else:
        return 'прочие цели'
data['purpose_group']=data['purpose'].apply(purpose_group)
data = data.drop(columns=['purpose'])
display(data.head()) 

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,total_income_category,purpose_group
0,1,8437,42.0,0,0,F,сотрудник,0,253875,B,операции с недвижимостью
1,1,4024,36.0,1,0,F,сотрудник,0,112080,C,операции с автомобилем
2,0,5623,33.0,1,0,M,сотрудник,0,145885,C,операции с недвижимостью
3,3,4124,32.0,1,0,M,сотрудник,0,267628,B,получение образования
4,0,340266,53.0,1,1,F,пенсионер,0,158616,C,проведение свадьбы


# Ответы на вопросы

## Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [46]:
data_pivot_children = data.pivot_table(index='children', values='debt', aggfunc=['sum','count'])
data_pivot_children.reset_index()
data_pivot_children['debt%']=data_pivot_children['sum']/data_pivot_children['count']
display(data_pivot_children)

,sum,count,debt%
,debt,debt,
children,,,
0,1063,14089,0.075449
1,445,4855,0.091658
2,202,2128,0.094925
3,27,330,0.081818
4,4,41,0.097561
5,0,9,0.000000


**Ответ**: Исходя из данных таблицы выше нет зависимости между количеством детей в семье и возвратом кредита в срок. 
*Несмотря на вывод выше, отдельно были подсчитаны доверительные интервалы для групп заемщиков с детьми (8,55%-9,87%) и без детей (7,11%-7,98%), и, так как интервалы не пересекаются, предварительно можно сделать вывод, что есть корреляция между наличием детей и задержкой в возврате кредита, хотя и нельзя говорить о прямой причинно-следственной связи.

## Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [47]:
data_pivot_family = data.pivot_table(index='family_status_id', values='debt', aggfunc=['sum','count'])
data_pivot_family.reset_index()
data_pivot_family['debt%']=data_pivot_family['sum']/data_pivot_family['count']
display(data_pivot_family)

,sum,count,debt%
,debt,debt,
family_status_id,,,
0,931,12339,0.075452
1,388,4149,0.093517
2,63,959,0.065693
3,85,1195,0.071130
4,274,2810,0.097509


**Ответ**: риск невозврата кредита в срок выше у групп с id 1 (**гражданский брак**) и 4 (**не женат/ не замужем**).

## Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [48]:
data_pivot_income = data.pivot_table(index='total_income_category', values='debt', aggfunc=['sum','count'])
data_pivot_income.reset_index()
data_pivot_income['debt%']=data_pivot_income['sum']/data_pivot_income['count']
display(data_pivot_income)

,sum,count,debt%
,debt,debt,
total_income_category,,,
A,2,25,0.080000
B,356,5041,0.070621
C,1360,16014,0.084926
D,21,350,0.060000
E,2,22,0.090909


**Ответ**: заемщики с уровнем дохода **D**, или от 30.001,0р до 50.000,0р, с наибольшей вероятностью возвращают кредит в срок. 
*Однако, следует отметить, что доверительный интервал у группы заемщиков **D** составляет 3,51% – 8,49%, поэтому для того, чтобы с увернностью утверждать о надежности заемщиков **D**, следует накопить больше данных о них. 

## Вопрос 4: Как разные цели кредита влияют на его возврат в срок?

In [49]:
data_pivot_purpose = data.pivot_table(index='purpose_group', values='debt', aggfunc=['sum','count'])
data_pivot_purpose.reset_index()
data_pivot_purpose['debt%']=data_pivot_purpose['sum']/data_pivot_purpose['count']
display(data_pivot_purpose)

,sum,count,debt%
,debt,debt,
purpose_group,,,
операции с автомобилем,403,4306,0.093590
операции с недвижимостью,782,10810,0.072340
получение образования,370,4013,0.092200
проведение свадьбы,186,2323,0.080069


**Ответ**: Заемщики чаще выплачивают кредиты в срок в том случае, если кредит взят под **операции с недвижимостью** и **проведение свадьбы**

# Общий вывод:

Проведя комплексное исследование надежности заемщиков, можем сделать вывод:
- по итогам анализа каждой из 4 гипотез были обнаружены зависимости,
- требуется изучить теорию о доверительных интервалах, чтобы лучше понимать, существует ли корреляция данных или нет.